In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [2]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df.loc[:, 'term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df.loc[:, 'length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df.loc[:, 'rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df.loc[:, 'room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df.loc[:, 'n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df.loc[:, 'weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df.loc[:, 'weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(HTML(interactions_df.head(15).to_html()))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several version of the prepare_users_df method and test which works best in your recommender.

In [3]:
def prepare_users_df(interactions_df):

    # Write your code here
    
    USR_users_df = pd.get_dummies(interactions_df[['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket' ,'weekend_stay']])
    USR_users_df['user_id'] = interactions_df['user_id']
    USR_temp_user_list = USR_users_df["user_id"].unique()
    
    #--TAB
    USER_user_id = []
    USER_term_WinterVacation = []
    USER_term_Easter = []
    USER_term_OffSeason = []
    USER_term_HighSeason = []
    USER_term_LowSeason = []
    USER_term_MayLongWeekend = []
    USER_term_NewYear = []
    USER_term_Christmas = []
    USER_length_of_stay_bucket_0_1 = []
    USER_length_of_stay_bucket_2_3 = []
    USER_length_of_stay_bucket_4_7 = []
    USER_length_of_stay_bucket_8 = []
    USER_rate_plan_Standard = []
    USER_rate_plan_Nonref = []
    USER_room_segment_0_160 = []
    USER_room_segment_160_260 = []
    USER_room_segment_260_360 = []
    USER_room_segment_360_500 = []
    USER_room_segment_500_900 = []
    USER_n_people_bucket_1 = []
    USER_n_people_bucket_2 = []
    USER_n_people_bucket_3 = []
    USER_n_people_bucket_5 = []
    USER_weekend_stay = []
    
    for x in USR_temp_user_list:
        sample = USR_users_df.loc[USR_users_df['user_id'] == x].groupby('user_id').sum()
        USER_user_id.append(x)
        
        sample = sample.values.tolist()
        sample = sample[0]
        
        A = (sample[8] + sample[9] + sample[10] + sample[11])/4
        B = (sample[12] + sample[13] ) /2
        C = (sample[14] + sample[15] + sample[16] + sample[17] + sample[18] ) / 5
        D = (sample[19] + sample[20] + sample[21] + sample[22] ) / 4
        
        val_tab = [0,0,0,0,0,0,0,0,A,A,A,A,B,B,C,C,C,C,C,D,D,D,D,0]    
        
        
        for i in range(len(sample[:-2])):
            if i < 8:
                if sample[i]>0:
                    val_tab[i]=1
            else:
                if sample[i]>val_tab[i]:
                    val_tab[i]=1
                else:
                    val_tab[i]=0
        
        if sample[-1] >= sample[-2]:
            val_tab[-1]=1
        

        USER_term_WinterVacation.append(val_tab[0])
        USER_term_Easter.append(val_tab[1])
        USER_term_OffSeason.append(val_tab[2])
        USER_term_HighSeason.append(val_tab[3])
        USER_term_LowSeason.append(val_tab[4])
        USER_term_MayLongWeekend.append(val_tab[5])
        USER_term_NewYear.append(val_tab[6])
        USER_term_Christmas.append(val_tab[7])
        USER_length_of_stay_bucket_0_1.append(val_tab[8])
        USER_length_of_stay_bucket_2_3.append(val_tab[9])
        USER_length_of_stay_bucket_4_7.append(val_tab[10])
        USER_length_of_stay_bucket_8.append(val_tab[11])
        USER_rate_plan_Standard.append(val_tab[12])
        USER_rate_plan_Nonref.append(val_tab[13])
        USER_room_segment_0_160.append(val_tab[14])
        USER_room_segment_160_260.append(val_tab[15])
        USER_room_segment_260_360.append(val_tab[16])
        USER_room_segment_360_500.append(val_tab[17])
        USER_room_segment_500_900.append(val_tab[18])
        USER_n_people_bucket_1.append(val_tab[19])
        USER_n_people_bucket_2.append(val_tab[20])
        USER_n_people_bucket_3.append(val_tab[21])
        USER_n_people_bucket_5.append(val_tab[22])
        USER_weekend_stay.append(val_tab[23])
        
        
    del USR_temp_user_list, USR_users_df, sample, val_tab
    
    users_df = {
        'user_id': USER_user_id, 
        'USER_term_WinterVacation': USER_term_WinterVacation,
        'USER_term_Easter': USER_term_Easter,
        'USER_term_OffSeason': USER_term_OffSeason,
        'USER_term_HighSeason': USER_term_HighSeason,
        'USER_term_LowSeason': USER_term_LowSeason,
        'USER_term_MayLongWeekend': USER_term_MayLongWeekend,
        'USER_term_NewYear': USER_term_NewYear,
        'USER_term_Christmas': USER_term_Christmas,
        'USER_length_of_stay_bucket_[0-1]': USER_length_of_stay_bucket_0_1,
        'USER_length_of_stay_bucket_[2-3]': USER_length_of_stay_bucket_2_3,
        'USER_length_of_stay_bucket_[4-7]': USER_length_of_stay_bucket_4_7,
        'USER_length_of_stay_bucket_[8-inf]': USER_length_of_stay_bucket_8,
        'USER_rate_plan_Standard': USER_rate_plan_Standard,
        'USER_rate_plan_Nonref': USER_rate_plan_Nonref,
        'USER_room_segment_[0-160]': USER_room_segment_0_160,
        'USER_room_segment_[160-260]': USER_room_segment_160_260,
        'USER_room_segment_[260-360]': USER_room_segment_260_360,
        'USER_room_segment_[360-500]': USER_room_segment_360_500,
        'USER_room_segment_[500-900]': USER_room_segment_500_900,
        'USER_n_people_bucket_[1-1]': USER_n_people_bucket_1,
        'USER_n_people_bucket_[2-2]': USER_n_people_bucket_2,
        'USER_n_people_bucket_[3-4]': USER_n_people_bucket_3,
        'USER_n_people_bucket_[5-inf]': USER_n_people_bucket_5,
        'USER_weekend_stay': USER_weekend_stay
    }
    
    del USER_term_WinterVacation, USER_term_Easter, USER_term_OffSeason, USER_term_HighSeason,USER_term_LowSeason
    del USER_term_MayLongWeekend, USER_term_NewYear, USER_term_Christmas
    del USER_length_of_stay_bucket_0_1, USER_length_of_stay_bucket_2_3, USER_length_of_stay_bucket_4_7, USER_length_of_stay_bucket_8
    del USER_rate_plan_Standard, USER_rate_plan_Nonref, USER_room_segment_0_160, USER_room_segment_160_260
    del USER_room_segment_360_500, USER_room_segment_500_900, USER_n_people_bucket_1, USER_n_people_bucket_2
    del USER_n_people_bucket_3, USER_n_people_bucket_5, USER_weekend_stay
    
    USR_user_features = []
    for key in users_df.keys():
        USR_user_features.append(key)
    
    user_features = USR_user_features
    
    del USR_user_features
     
    
    users_df = pd.DataFrame (users_df, columns = user_features)
        
    users_df.set_index('user_id')
    
    return users_df, user_features[1:]
    


users_df, user_features = prepare_users_df(interactions_df)

print(user_features)
display(HTML(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15).to_html()))

['USER_term_WinterVacation', 'USER_term_Easter', 'USER_term_OffSeason', 'USER_term_HighSeason', 'USER_term_LowSeason', 'USER_term_MayLongWeekend', 'USER_term_NewYear', 'USER_term_Christmas', 'USER_length_of_stay_bucket_[0-1]', 'USER_length_of_stay_bucket_[2-3]', 'USER_length_of_stay_bucket_[4-7]', 'USER_length_of_stay_bucket_[8-inf]', 'USER_rate_plan_Standard', 'USER_rate_plan_Nonref', 'USER_room_segment_[0-160]', 'USER_room_segment_[160-260]', 'USER_room_segment_[260-360]', 'USER_room_segment_[360-500]', 'USER_room_segment_[500-900]', 'USER_n_people_bucket_[1-1]', 'USER_n_people_bucket_[2-2]', 'USER_n_people_bucket_[3-4]', 'USER_n_people_bucket_[5-inf]', 'USER_weekend_stay']


,user_id,USER_term_WinterVacation,USER_term_Easter,USER_term_OffSeason,USER_term_HighSeason,USER_term_LowSeason,USER_term_MayLongWeekend,USER_term_NewYear,USER_term_Christmas,USER_length_of_stay_bucket_[0-1],USER_length_of_stay_bucket_[2-3],USER_length_of_stay_bucket_[4-7],USER_length_of_stay_bucket_[8-inf],USER_rate_plan_Standard,USER_rate_plan_Nonref,USER_room_segment_[0-160],USER_room_segment_[160-260],USER_room_segment_[260-360],USER_room_segment_[360-500],USER_room_segment_[500-900],USER_n_people_bucket_[1-1],USER_n_people_bucket_[2-2],USER_n_people_bucket_[3-4],USER_n_people_bucket_[5-inf],USER_weekend_stay
0,1,1,0,1,1,1,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0
47,50,1,0,1,1,1,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,1,0
92,96,1,0,1,1,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0
135,115,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1
645,706,1,0,1,1,1,1,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,0
1603,1736,1,0,1,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,1
7296,7779,1,0,1,1,1,0,0,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,0


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into on-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because of the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features.

In [4]:
def prepare_items_df(interactions_df):
    
    # Write your code here
    
    
    ITEM_items_df = pd.get_dummies(interactions_df[['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket' ,'weekend_stay']])
    ITEM_items_df['item_id'] = interactions_df['item_id']
    ITEM_temp_item_list = ITEM_items_df["item_id"].unique()
    
    #--TAB
    ITEM_item_id = []    
    ITEM_term_WinterVacation = []
    ITEM_term_Easter = []
    ITEM_term_OffSeason = []
    ITEM_term_HighSeason = []
    ITEM_term_LowSeason = []
    ITEM_term_MayLongWeekend = []
    ITEM_term_NewYear = []
    ITEM_term_Christmas = []
    ITEM_length_of_stay_bucket_0_1 = []
    ITEM_length_of_stay_bucket_2_3 = []
    ITEM_length_of_stay_bucket_4_7 = []
    ITEM_length_of_stay_bucket_8 = []
    ITEM_rate_plan_Standard = []
    ITEM_rate_plan_Nonref = []
    ITEM_room_segment_0_160 = []
    ITEM_room_segment_160_260 = []
    ITEM_room_segment_260_360 = []
    ITEM_room_segment_360_500 = []
    ITEM_room_segment_500_900 = []
    ITEM_n_people_bucket_1 = []
    ITEM_n_people_bucket_2 = []
    ITEM_n_people_bucket_3 = []
    ITEM_n_people_bucket_5 = []
    ITEM_weekend_stay = []
    
    for x in ITEM_temp_item_list:
        sample = ITEM_items_df.loc[ITEM_items_df['item_id'] == x].groupby('item_id').sum()
        ITEM_item_id.append(x)
        
        sample = sample.values.tolist()
        sample = sample[0]
        
        A = (sample[8] + sample[9] + sample[10] + sample[11])/4
        B = (sample[12] + sample[13] ) /2
        C = (sample[14] + sample[15] + sample[16] + sample[17] + sample[18] ) / 5
        D = (sample[19] + sample[20] + sample[21] + sample[22] ) / 4
        
        val_tab = [0,0,0,0,0,0,0,0,A,A,A,A,B,B,C,C,C,C,C,D,D,D,D,0]    
        
        
        for i in range(len(sample[:-2])):
            if i < 8:
                if sample[i]>0:
                    val_tab[i]=1
            else:
                if sample[i]>val_tab[i]:
                    val_tab[i]=1
                else:
                    val_tab[i]=0
        
        if sample[-1] >= sample[-2]:
            val_tab[-1]=1
        

        
        ITEM_term_WinterVacation.append(val_tab[0])
        ITEM_term_Easter.append(val_tab[1])
        ITEM_term_OffSeason.append(val_tab[2])
        ITEM_term_HighSeason.append(val_tab[3])
        ITEM_term_LowSeason.append(val_tab[4])
        ITEM_term_MayLongWeekend.append(val_tab[5])
        ITEM_term_NewYear.append(val_tab[6])
        ITEM_term_Christmas.append(val_tab[7])
        ITEM_length_of_stay_bucket_0_1.append(val_tab[8])
        ITEM_length_of_stay_bucket_2_3.append(val_tab[9])
        ITEM_length_of_stay_bucket_4_7.append(val_tab[10])
        ITEM_length_of_stay_bucket_8.append(val_tab[11])
        ITEM_rate_plan_Standard.append(val_tab[12])
        ITEM_rate_plan_Nonref.append(val_tab[13])
        ITEM_room_segment_0_160.append(val_tab[14])
        ITEM_room_segment_160_260.append(val_tab[15])
        ITEM_room_segment_260_360.append(val_tab[16])
        ITEM_room_segment_360_500.append(val_tab[17])
        ITEM_room_segment_500_900.append(val_tab[18])
        ITEM_n_people_bucket_1.append(val_tab[19])
        ITEM_n_people_bucket_2.append(val_tab[20])
        ITEM_n_people_bucket_3.append(val_tab[21])
        ITEM_n_people_bucket_5.append(val_tab[22])
        ITEM_weekend_stay.append(val_tab[23])
        
        
    del ITEM_temp_item_list, ITEM_items_df, sample, val_tab
    
    items_df = {
        'item_id': ITEM_item_id,
        'ITEM_term_WinterVacation': ITEM_term_WinterVacation,
        'ITEM_term_Easter': ITEM_term_Easter,
        'ITEM_term_OffSeason': ITEM_term_OffSeason,
        'ITEM_term_HighSeason': ITEM_term_HighSeason,
        'ITEM_term_LowSeason': ITEM_term_LowSeason,
        'ITEM_term_MayLongWeekend': ITEM_term_MayLongWeekend,
        'ITEM_term_NewYear': ITEM_term_NewYear,
        'ITEM_term_Christmas': ITEM_term_Christmas,
        'ITEM_length_of_stay_bucket_[0-1]': ITEM_length_of_stay_bucket_0_1,
        'ITEM_length_of_stay_bucket_[2-3]': ITEM_length_of_stay_bucket_2_3,
        'ITEM_length_of_stay_bucket_[4-7]': ITEM_length_of_stay_bucket_4_7,
        'ITEM_length_of_stay_bucket_[8-inf]': ITEM_length_of_stay_bucket_8,
        'ITEM_rate_plan_Standard': ITEM_rate_plan_Standard,
        'ITEM_rate_plan_Nonref': ITEM_rate_plan_Nonref,
        'ITEM_room_segment_[0-160]': ITEM_room_segment_0_160,
        'ITEM_room_segment_[160-260]': ITEM_room_segment_160_260,
        'ITEM_room_segment_[260-360]': ITEM_room_segment_260_360,
        'ITEM_room_segment_[360-500]': ITEM_room_segment_360_500,
        'ITEM_room_segment_[500-900]': ITEM_room_segment_500_900,
        'ITEM_n_people_bucket_[1-1]': ITEM_n_people_bucket_1,
        'ITEM_n_people_bucket_[2-2]': ITEM_n_people_bucket_2,
        'ITEM_n_people_bucket_[3-4]': ITEM_n_people_bucket_3,
        'ITEM_n_people_bucket_[5-inf]': ITEM_n_people_bucket_5,
        'ITEM_weekend_stay': ITEM_weekend_stay
    }
    
    del ITEM_term_WinterVacation, ITEM_term_Easter, ITEM_term_OffSeason, ITEM_term_HighSeason,ITEM_term_LowSeason
    del ITEM_term_MayLongWeekend, ITEM_term_NewYear, ITEM_term_Christmas
    del ITEM_length_of_stay_bucket_0_1, ITEM_length_of_stay_bucket_2_3, ITEM_length_of_stay_bucket_4_7, ITEM_length_of_stay_bucket_8
    del ITEM_rate_plan_Standard, ITEM_rate_plan_Nonref, ITEM_room_segment_0_160, ITEM_room_segment_160_260
    del ITEM_room_segment_360_500, ITEM_room_segment_500_900, ITEM_n_people_bucket_1, ITEM_n_people_bucket_2
    del ITEM_n_people_bucket_3, ITEM_n_people_bucket_5, ITEM_weekend_stay
    
    ITEM_item_features = []
    for key in items_df.keys():
        ITEM_item_features.append(key)
    
    item_features = ITEM_item_features
    
    del ITEM_item_features
    
    
    items_df = pd.DataFrame (items_df, columns = item_features)

    
        
    items_df.set_index('item_id')
    
    return items_df, item_features[1:]



#display(HTML(interactions_df.head(15).to_html()))

items_df, item_features = prepare_items_df(interactions_df)

print(item_features)

display(HTML(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15).to_html()))

['ITEM_term_WinterVacation', 'ITEM_term_Easter', 'ITEM_term_OffSeason', 'ITEM_term_HighSeason', 'ITEM_term_LowSeason', 'ITEM_term_MayLongWeekend', 'ITEM_term_NewYear', 'ITEM_term_Christmas', 'ITEM_length_of_stay_bucket_[0-1]', 'ITEM_length_of_stay_bucket_[2-3]', 'ITEM_length_of_stay_bucket_[4-7]', 'ITEM_length_of_stay_bucket_[8-inf]', 'ITEM_rate_plan_Standard', 'ITEM_rate_plan_Nonref', 'ITEM_room_segment_[0-160]', 'ITEM_room_segment_[160-260]', 'ITEM_room_segment_[260-360]', 'ITEM_room_segment_[360-500]', 'ITEM_room_segment_[500-900]', 'ITEM_n_people_bucket_[1-1]', 'ITEM_n_people_bucket_[2-2]', 'ITEM_n_people_bucket_[3-4]', 'ITEM_n_people_bucket_[5-inf]', 'ITEM_weekend_stay']


,item_id,ITEM_term_WinterVacation,ITEM_term_Easter,ITEM_term_OffSeason,ITEM_term_HighSeason,ITEM_term_LowSeason,ITEM_term_MayLongWeekend,ITEM_term_NewYear,ITEM_term_Christmas,ITEM_length_of_stay_bucket_[0-1],ITEM_length_of_stay_bucket_[2-3],ITEM_length_of_stay_bucket_[4-7],ITEM_length_of_stay_bucket_[8-inf],ITEM_rate_plan_Standard,ITEM_rate_plan_Nonref,ITEM_room_segment_[0-160],ITEM_room_segment_[160-260],ITEM_room_segment_[260-360],ITEM_room_segment_[360-500],ITEM_room_segment_[500-900],ITEM_n_people_bucket_[1-1],ITEM_n_people_bucket_[2-2],ITEM_n_people_bucket_[3-4],ITEM_n_people_bucket_[5-inf],ITEM_weekend_stay
0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0
1,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0
2,2,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1
3,3,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0
4,4,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0
5,5,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0
6,6,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object for later use in the recommend method. Overwrite the users_df variable. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real interactions and equal to 0 for those newly added interactions. Generate several negative interactions per every positive interactions (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and use initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, Neural Networks or any model of your choice.
  - A different approach where you treat each item as a class, you train directly on categorical features of items and users (you would have to design appropriate categorical features for users) and you fit classifiers (e.g. Decision Tree classifier, Naive Bayes classifier etc.) instead of regressors.

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]
        
        
        # Generate negative interactions
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
        
        negative_interactions = []
        
#START----------------------------------------------------------------------------------------------------------------------
        # Write your code here
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions 
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.
        
        
        #ZMIENNE DF CHWILOWE
        USR_temp_user_list = interactions_df["user_id"].unique()
        USR_temp_item_list = interactions_df["item_id"].unique()
        USR_temp_df = interactions_df[["user_id", "item_id"]]
        
        #ZMIENNE LISTY
        USR_us = []
        USR_it = []
        for USR_user in USR_temp_user_list:
            for USR_item in USR_temp_item_list:
                USR_us.append(USR_user)
                USR_it.append(USR_item)

        del USR_temp_user_list, USR_temp_item_list
        
        #SŁOWNIK
        USR_new_df = {
            'user_id': USR_us,
            'item_id': USR_it
        }
        
        del USR_us, USR_it
        
        #ZMINA ZŁOWNIKA NA DF
        USR_new_df = pd.DataFrame(USR_new_df, columns = ['user_id','item_id'])
    
        #USUNIECIE Z NOWEGO SŁOWNIKA ELEMENTOW NIE BEDĄCYCH NEG_IT
        USR_new_df = pd.merge(USR_new_df, USR_temp_df, on=['user_id','item_id'], how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)
        
        del USR_temp_df
        
        #WYBRANIE ODPOWIEDNIEJ ILOŚCI NEG_IT = len(interactions_df)*self.n_neg_per_pos
        USR_df_subset = USR_new_df.sample(len(interactions_df)*self.n_neg_per_pos)
        
        del USR_new_df
        
        #UTWORZENIE KROTEK Z WYBRANYCH ELEMENTÓW
        USR_tuples = [tuple(USR_x) for USR_x in USR_df_subset.values]
        
        del USR_df_subset
        
        #ZMIANA KROTEK NA LISTY, DODANIE 0 I ZAMIANA NA KROTKI ABY DODAC DO LISTY
        for USR_i in range(len(USR_tuples)):
            USR_lista = list(USR_tuples[USR_i])
            USR_lista.append(0)
            negative_interactions.append(tuple(USR_lista))
            
        del USR_tuples, USR_lista

        
#END------------------------------------------------------------------------------------------------------------------------        
        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values
    
        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
    
#START----------------------------------------------------------------------------------------------------------------------
        # Write your code here
        # Prepare users_df and items_df
        # For users_df you just need to merge user features from self.users_df to users_df 
        # (the users for which you generate recommendations)
        # For items you have to apply the prepare_items_df method to items_df.
        
        # Score the items
        
        users_df = pd.merge(users_df, self.users_df, on=['user_id'])
        
        items_df, item_features = prepare_items_df(items_df)

#END------------------------------------------------------------------------------------------------------------------------        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():

#START----------------------------------------------------------------------------------------------------------------------
            # Write your code here
            # Create a Carthesian product of users from users_df and items from items_df
            
            USER_temp_user = users_df.loc[users_df['user_id'] == user['user_id']]
            
            print(type(USER_temp_user))
            print(type(items_df))
            Carthesian = pd.merge(USER_temp_user, items_df, how='cross')
            del USER_temp_user
#END------------------------------------------------------------------------------------------------------------------------        

#START----------------------------------------------------------------------------------------------------------------------
            # Write your code here
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            
            scores = self.model.predict(Carthesian)
            print(Carthesian)
            print(scores)
            
#END------------------------------------------------------------------------------------------------------------------------        
            
#START----------------------------------------------------------------------------------------------------------------------
            # Write your code here
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.
            chosen_ids = None
            
            
            #chosen_ids = 
            
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
#END------------------------------------------------------------------------------------------------------------------------        
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [6]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [7]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [8]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(HTML(recommendations.to_html()))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

# Tuning method

In [ ]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [ ]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(HTML(results.to_html()))
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [ ]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [ ]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(
    **{'n_neg_per_pos': 7})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(linear_cbui_tts_results.to_html()))

In [ ]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(amazon_tts_results.to_html()))

In [ ]:
tts_results = pd.concat([linear_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(HTML(tts_results.to_html()))